<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENTIC_GLM4DOT6_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://z.ai/manage-apikey/subscription

In [1]:
from google.colab import userdata
ZAI_KEY_API = userdata.get('ZAI_KEY')


In [ ]:
!pip install pulp -q

In [ ]:
!pip install pulp==2.9.0 -q

In [22]:
!pip show pulp

Name: PuLP
Version: 2.9.0
Summary: PuLP is an LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems.
Home-page: https://github.com/coin-or/pulp
Author: J.S. Roy and S.A. Mitchell and F. Peschiera
Author-email: pulp@stuartmitchell.com
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: 
Required-by: 


In [ ]:
!pip install flake8 -q

In [21]:
!pip show flake8

Name: flake8
Version: 7.3.0
Summary: the modular source code checker: pep8 pyflakes and co
Home-page: https://github.com/pycqa/flake8
Author: Tarek Ziade
Author-email: tarek@ziade.org
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: mccabe, pycodestyle, pyflakes
Required-by: 


In [20]:
import warnings

# Suppress all UserWarnings from the pulp package
warnings.filterwarnings(
    "ignore",
    message="Overwriting previously set objective.",
    category=UserWarning,
    module='pulp'
)

# FINAL

## PART1

Part 1: Imports, Configuration, and Tool Definitions

This part includes all imports, logging setup, API key configuration, and tool functions. These are unchanged from the previous version, as they are working correctly (e.g., handling mock transient failures with fallbacks and avoiding JSONDecodeError).

In [28]:
import json
import time
import logging
import random
import re
from typing import Dict, List, Any
from openai import OpenAI
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import plotly.graph_objects as go
from tenacity import retry, stop_after_attempt, wait_exponential

# --- 0. Configuration (Logging & API Keys) ---
logging.basicConfig(
    filename='aoc_agent.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

try:
    from google.colab import userdata
    ZAI_KEY_API = userdata.get('ZAI_KEY')
    if not ZAI_KEY_API:
        raise ValueError("ZAI_KEY not found in Colab secrets.")
except Exception as e:
    logger.error(f"Error loading API key: {e}")
    print(f"Error loading API key: {e}. Please ensure 'ZAI_KEY' is set in Colab Secrets.")
    ZAI_KEY_API = "PLACEHOLDER_ERROR_KEY"

# Initialize Z.ai Client
client = OpenAI(
    api_key=ZAI_KEY_API,
    base_url="https://api.z.ai/api/paas/v4/"
)

# --- 1. Tool Definitions ---

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def get_flight_status(flight_number: str, departure_icao: str = None) -> str:
    logger.info(f"Fetching flight status for {flight_number}")
    mock_data = {
        "AA123": {
            "flight_number": "AA123",
            "status": "delayed",
            "delay_minutes": 45,
            "eta": "2025-10-01T18:30:00Z",
            "gate": "B12",
            "aircraft": "B737",
            "departure_icao": "KJFK",
            "destination_icao": "KLAX"
        },
        "UA456": {
            "flight_number": "UA456",
            "status": "on_time",
            "delay_minutes": 0,
            "eta": "2025-10-01T17:45:00Z",
            "gate": "C5",
            "aircraft": "A320",
            "departure_icao": "KORD",
            "destination_icao": "KSFO"
        },
        "DL789": {
            "flight_number": "DL789",
            "status": "delayed",
            "delay_minutes": 60,
            "eta": "2025-10-01T19:00:00Z",
            "gate": "A10",
            "aircraft": "A321",
            "departure_icao": "KJFK",
            "destination_icao": "KORD"
        }
    }

    # *** SIMULATED FAILURE BLOCK REMOVED HERE ***
    # if random.random() < 0.05:
    #     logger.warning(f"Simulating transient failure for {flight_number}")
    #     raise Exception("Mock transient failure")

    data = mock_data.get(flight_number, {
        "flight_number": flight_number,
        "status": "unknown",
        "delay_minutes": 0,
        "eta": "Unknown",
        "gate": "Unknown",
        "aircraft": "Unknown",
        "departure_icao": departure_icao or "Unknown",
        "destination_icao": "Unknown"
    })
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def get_aviation_weather(icao_code: str) -> str:
    logger.info(f"Fetching weather for {icao_code}")
    if not isinstance(icao_code, str) or not icao_code:
        logger.warning(f"Invalid icao_code: {icao_code}, defaulting to KJFK")
        icao_code = "KJFK"
    mock_data = {
        "KJFK": {
            "icao": "KJFK",
            "metar": "KJFK 011730Z 27010KT 10SM FEW030 BKN050 15/10 A2992",
            "conditions": "Light wind, clear skies"
        },
        "KLAX": {
            "icao": "KLAX",
            "metar": "KLAX 011745Z 18008KT 5SM HZ BKN040 22/18 A2985",
            "conditions": "Haze, moderate visibility"
        },
        "KORD": {
            "icao": "KORD",
            "metar": "KORD 011800Z 30012KT 8SM OVC035 12/08 A2990",
            "conditions": "Overcast, windy"
        }
    }

    # *** SIMULATED FAILURE BLOCK REMOVED HERE ***
    # if random.random() < 0.05:
    #     logger.warning(f"Simulating transient failure for {icao_code}")
    #     raise Exception("Mock transient failure")

    data = mock_data.get(icao_code, {
        "icao": icao_code,
        "metar": f"{icao_code} 011730Z 27010KT 10SM CLR 20/15 A2995",
        "conditions": "Assumed clear conditions (fallback)"
    })
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def check_crew_availability(flight_number: str) -> str:
    logger.info(f"Checking crew availability for {flight_number}")
    mock_data = {
        "AA123": {
            "available": True,
            "pilot": "Available (ID: P001, Rest: 10h)",
            "copilot": "Available (ID: C001, Rest: 9h)",
            "cabin_crew": ["Available (ID: CC001, Rest: 8h)", "Available (ID: CC002, Rest: 8h)"]
        },
        "UA456": {
            "available": False,
            "pilot": "Unavailable (ID: P002, Rest: 2h)",
            "copilot": "Available (ID: C002, Rest: 7h)",
            "cabin_crew": ["Available (ID: CC003, Rest: 6h)"]
        }
    }
    data = mock_data.get(flight_number, {
        "available": False,
        "pilot": "Unknown",
        "copilot": "Unknown",
        "cabin_crew": []
    })
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def estimate_passenger_impact(flight_number: str, delay_minutes: int) -> str:
    logger.info(f"Estimating passenger impact for {flight_number}, delay: {delay_minutes}")
    passengers = random.randint(100, 200)
    rebooking_cost = 200 * passengers if delay_minutes > 60 else 0
    compensation_cost = 0
    total_cost = rebooking_cost + compensation_cost
    data = {
        "flight_number": flight_number,
        "passengers": passengers,
        "rebooking_cost": rebooking_cost,
        "compensation_cost": compensation_cost,
        "total_cost": total_cost
    }
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def predict_delay_escalation(flight_number: str, delay_minutes: int) -> str:
    logger.info(f"Predicting delay escalation for {flight_number}, delay: {delay_minutes}")
    likelihood = min(0.1 + delay_minutes * 0.005, 0.9)
    additional_delay = max(30, delay_minutes // 2)
    data = {
        "escalation_likelihood": likelihood,
        "predicted_additional_delay": additional_delay
    }
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def check_regulatory_compliance(flight_data: str, crew_data: str) -> str:
    logger.info("Checking regulatory compliance")
    try:
        flight = json.loads(flight_data)
        crew = json.loads(crew_data)
        issues = []
        if not crew.get("available", False):
            issues.append("Crew rest non-compliant")
        if flight.get("delay_minutes", 0) > 180:
            issues.append("Delay exceeds regulatory threshold")
        data = {
            "status": "Compliant" if not issues else "Non-compliant",
            "issues": issues
        }
        return json.dumps(data)
    except json.JSONDecodeError as e:
        logger.error(f"JSONDecodeError in check_regulatory_compliance: {str(e)}")
        return json.dumps({"error": f"Invalid input data: {str(e)}"})

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def estimate_passenger_feedback(flight_number: str, delay_minutes: int) -> str:
    logger.info(f"Estimating passenger feedback for {flight_number}, delay: {delay_minutes}")
    satisfaction = max(0.1, 0.9 - delay_minutes * 0.005)
    feedback = "Positive" if satisfaction > 0.7 else "Neutral" if satisfaction > 0.4 else "Negative"
    data = {
        "flight_number": flight_number,
        "satisfaction_score": satisfaction,
        "feedback": feedback
    }
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def get_atc_update(flight_number: str) -> str:
    logger.info(f"Fetching ATC update for {flight_number}")
    data = {
        "status": "Clearance expected in 30 min",
        "additional_delay_min": 30
    }
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def calculate_cost_threshold(flight_data: str, passenger_data: str, prediction_data: str) -> str:
    logger.info("Calculating cost threshold")
    try:
        flight = json.loads(flight_data)
        passenger = json.loads(passenger_data)
        prediction = json.loads(prediction_data)
        delay_min = flight.get("delay_minutes", 0)
        cancel_cost = 50000 + passenger.get("passengers", 100) * 300
        delay_cost = delay_min * 250 + passenger.get("total_cost", 0)
        data = {
            "cancel_cost": cancel_cost,
            "delay_cost": delay_cost,
            "threshold_min": 100,
            "recommendation": "Cancel" if delay_min > 180 else "Delay"
        }
        return json.dumps(data)
    except json.JSONDecodeError as e:
        logger.error(f"JSONDecodeError in calculate_cost_threshold: {str(e)}")
        return json.dumps({"error": f"Invalid input data: {str(e)}"})

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def pre_plan_crew_swap(flight_number: str, crew_data: str, delay_minutes: int) -> str:
    logger.info(f"Pre-planning crew swap for {flight_number}, delay: {delay_minutes}")
    try:
        crew = json.loads(crew_data)
        recommendation = "No swap needed" if crew.get("available", False) else "Swap required"
        data = {
            "recommendation": recommendation,
            "standby_crew": None if crew.get("available", False) else {"pilot": "P002", "copilot": "C002"}
        }
        return json.dumps(data)
    except json.JSONDecodeError as e:
        logger.error(f"JSONDecodeError in pre_plan_crew_swap: {str(e)}")
        return json.dumps({"error": f"Invalid input data: {str(e)}"})

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def get_alternate_routes(flight_number: str, departure_icao: str, destination_icao: str) -> str:
    logger.info(f"Fetching alternate routes for {flight_number} from {departure_icao} to {destination_icao}")
    data = {
        "original_route": f"{departure_icao}-{destination_icao} direct",
        "alternates": [
            {"route": f"{departure_icao}-KPHL-{destination_icao}", "extra_time_min": 20, "extra_cost": 3000},
            {"route": f"{departure_icao}-KORD-{destination_icao}", "extra_time_min": 45, "extra_cost": 5000}
        ]
    }
    return json.dumps(data)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def update_passenger_feedback(flight_number: str, satisfaction_score: float, action: str) -> str:
    logger.info(f"Updating passenger feedback for {flight_number}, action: {action}")
    new_score = min(0.95, satisfaction_score + 0.1 if "amenities" in action.lower() else satisfaction_score)
    feedback = "Positive" if new_score > 0.7 else "Neutral" if new_score > 0.4 else "Negative"
    data = {
        "flight_number": flight_number,
        "satisfaction_score": new_score,
        "feedback": feedback
    }
    return json.dumps(data)

def suggest_operations_decision(
    flight_data: str, weather_data: str, crew_data: str, route_data: str,
    passenger_data: str, prediction_data: str, compliance_data: str,
    feedback_data: str, atc_data: str, threshold_data: str, priority: str
) -> str:
    logger.info(f"Suggesting operations decision with priority: {priority}")
    try:
        flight = json.loads(flight_data)
        weather = json.loads(weather_data)
        crew = json.loads(crew_data)
        routes = json.loads(route_data)
        passenger = json.loads(passenger_data)
        prediction = json.loads(prediction_data)
        compliance = json.loads(compliance_data)
        feedback = json.loads(feedback_data)
        atc = json.loads(atc_data)
        threshold = json.loads(threshold_data)

        delay_min = flight.get("delay_minutes", 0)
        if delay_min == 0:
            return json.dumps({
                "recommendation": [{"action": "Proceed on schedule", "priority": 1, "risk": "low"}],
                "estimated_cost": "$0",
                "risk_level": "low"
            })

        recommendation = []
        total_cost = 0
        risk_level = "medium" if prediction.get("escalation_likelihood", 0.5) > 0.3 else "low"

        if priority == "cost" and delay_min >= 120 and routes.get("alternates"):
            recommendation.append({
                "action": f"Reroute via {routes['alternates'][0]['route']} (+{routes['alternates'][0]['extra_time_min']} min)",
                "priority": 1,
                "risk": "medium",
                "cost": f"${routes['alternates'][0]['extra_cost']}"
            })
            recommendation.append({
                "action": f"Notify passengers: Reroute via {routes['alternates'][0]['route']}",
                "priority": 2,
                "risk": "low",
                "cost": "$1500"
            })
            recommendation.append({
                "action": "Proactive connection management at destination",
                "priority": 3,
                "risk": "medium",
                "cost": f"${passenger.get('total_cost', 0)}"
            })
            total_cost = routes["alternates"][0]["extra_cost"] + 1500 + passenger.get("total_cost", 0)
        elif priority == "passenger" and delay_min > 0:
            recommendation.append({
                "action": f"Delay {delay_min} min and provide amenities",
                "priority": 1,
                "risk": "medium" if prediction.get("escalation_likelihood", 0.5) > 0.3 else "low",
                "cost": "$3150" if delay_min > 90 else "$900"
            })
            recommendation.append({
                "action": f"Notify passengers: Delay {delay_min} min",
                "priority": 2,
                "risk": "low",
                "cost": "$100"
            })
            recommendation.append({
                "action": "Monitor ATC updates",
                "priority": 3,
                "risk": "low",
                "cost": "$0"
            })
            total_cost = (3150 if delay_min > 90 else 900) + 100
        else:
            recommendation.append({
                "action": f"Delay {delay_min} min and monitor",
                "priority": 1,
                "risk": "medium",
                "cost": "$0"
            })
            total_cost = passenger.get("total_cost", 0)

        return json.dumps({
            "recommendation": recommendation,
            "estimated_cost": f"${total_cost}",
            "risk_level": risk_level
        })
    except json.JSONDecodeError as e:
        logger.error(f"JSONDecodeError in suggest_operations_decision: {str(e)}")
        return json.dumps({"error": f"Invalid input data: {str(e)}"})

## PART2

Part 2: Tool Schemas and Available Functions

This part defines the tool schemas and the mapping of available functions. These are unchanged, as they correctly specify the tools and their parameters, and the output shows all tools executed successfully.

In [29]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flight_status",
            "description": "Retrieve real-time flight status by flight number.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number, e.g., AA123"},
                    "departure_icao": {"type": "string", "description": "ICAO code of departure airport, optional"}
                },
                "required": ["flight_number"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_aviation_weather",
            "description": "Fetch weather data for an airport by ICAO code.",
            "parameters": {
                "type": "object",
                "properties": {
                    "icao_code": {"type": "string", "description": "ICAO code, e.g., KJFK"}
                },
                "required": ["icao_code"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_crew_availability",
            "description": "Check crew availability for a flight.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number, e.g., AA123"}
                },
                "required": ["flight_number"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "estimate_passenger_impact",
            "description": "Estimate passenger impact due to delay.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number"},
                    "delay_minutes": {"type": "integer", "description": "Delay duration in minutes"}
                },
                "required": ["flight_number", "delay_minutes"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "predict_delay_escalation",
            "description": "Predict likelihood of delay escalation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number"},
                    "delay_minutes": {"type": "integer", "description": "Current delay in minutes"}
                },
                "required": ["flight_number", "delay_minutes"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_regulatory_compliance",
            "description": "Check compliance with aviation regulations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_data": {"type": "string", "description": "JSON string of flight data"},
                    "crew_data": {"type": "string", "description": "JSON string of crew data"}
                },
                "required": ["flight_data", "crew_data"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "estimate_passenger_feedback",
            "description": "Estimate passenger feedback based on delay.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number"},
                    "delay_minutes": {"type": "integer", "description": "Delay duration in minutes"}
                },
                "required": ["flight_number", "delay_minutes"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_atc_update",
            "description": "Fetch ATC updates for a flight.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number"}
                },
                "required": ["flight_number"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_cost_threshold",
            "description": "Calculate cost thresholds for delay vs. cancellation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_data": {"type": "string", "description": "JSON string of flight data"},
                    "passenger_data": {"type": "string", "description": "JSON string of passenger data"},
                    "prediction_data": {"type": "string", "description": "JSON string of prediction data"}
                },
                "required": ["flight_data", "passenger_data", "prediction_data"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "pre_plan_crew_swap",
            "description": "Pre-plan crew swap if needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number"},
                    "crew_data": {"type": "string", "description": "JSON string of crew data"},
                    "delay_minutes": {"type": "integer", "description": "Delay duration in minutes"}
                },
                "required": ["flight_number", "crew_data", "delay_minutes"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_alternate_routes",
            "description": "Fetch alternate routes for a flight.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number"},
                    "departure_icao": {"type": "string", "description": "ICAO code of departure"},
                    "destination_icao": {"type": "string", "description": "ICAO code of destination"}
                },
                "required": ["flight_number", "departure_icao", "destination_icao"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "suggest_operations_decision",
            "description": "Suggest operational decisions based on data.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_data": {"type": "string", "description": "JSON string of flight data"},
                    "weather_data": {"type": "string", "description": "JSON string of weather data"},
                    "crew_data": {"type": "string", "description": "JSON string of crew data"},
                    "route_data": {"type": "string", "description": "JSON string of route data"},
                    "passenger_data": {"type": "string", "description": "JSON string of passenger data"},
                    "prediction_data": {"type": "string", "description": "JSON string of prediction data"},
                    "compliance_data": {"type": "string", "description": "JSON string of compliance data"},
                    "feedback_data": {"type": "string", "description": "JSON string of feedback data"},
                    "atc_data": {"type": "string", "description": "JSON string of ATC data"},
                    "threshold_data": {"type": "string", "description": "JSON string of threshold data"},
                    "priority": {"type": "string", "description": "Priority: cost, passenger, or operation"}
                },
                "required": [
                    "flight_data", "weather_data", "crew_data", "route_data", "passenger_data",
                    "prediction_data", "compliance_data", "feedback_data", "atc_data", "threshold_data", "priority"
                ]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "update_passenger_feedback",
            "description": "Update passenger feedback based on action taken.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {"type": "string", "description": "Flight number"},
                    "satisfaction_score": {"type": "number", "description": "Current satisfaction score"},
                    "action": {"type": "string", "description": "Action taken, e.g., provide amenities"}
                },
                "required": ["flight_number", "satisfaction_score", "action"]
            }
        }
    }
]

available_functions = {
    "get_flight_status": get_flight_status,
    "get_aviation_weather": get_aviation_weather,
    "check_crew_availability": check_crew_availability,
    "estimate_passenger_impact": estimate_passenger_impact,
    "predict_delay_escalation": predict_delay_escalation,
    "check_regulatory_compliance": check_regulatory_compliance,
    "estimate_passenger_feedback": estimate_passenger_feedback,
    "get_atc_update": get_atc_update,
    "calculate_cost_threshold": calculate_cost_threshold,
    "pre_plan_crew_swap": pre_plan_crew_swap,
    "get_alternate_routes": get_alternate_routes,
    "suggest_operations_decision": suggest_operations_decision,
    "update_passenger_feedback": update_passenger_feedback
}

## PART3

Part 3: AOCCAgent Class with Corrected Methods

This part contains the AOCCAgent class with the updated update_context and reflect methods to fix the priority detection issue for the 120-minute delay scenario. The run_query and validate_tool_output methods remain unchanged, as they are functioning correctly (e.g., no JSONDecodeError or RetryError in the output). The update_context method now uses a more robust regex to detect "prioritize cost" and ensures the delay duration is updated correctly. The reflect method aligns with the cost priority by selecting rerouting actions for delays ≥ 120 minutes.

In [30]:
class AOCCAgent:
    def __init__(self):
        self.client = client
        self.conversation_history = []
        self.metrics = {"tools_called": 0, "response_time": 0.0, "errors": []}
        self.context = {
            "flight_number": None,
            "delay_minutes": None,
            "crew_available": None,
            "passenger_cost": 0,
            "escalation_likelihood": 0.5,
            "satisfaction_score": 0.5,
            "atc_status": "No update",
            "atc_delay": 0,
            "threshold_recommendation": "Delay",
            "departure_icao": None,
            "destination_icao": None,
            "priority": "passenger"
        }

    def validate_tool_output(self, output, tool_name):
        logger.info(f"Validating output for {tool_name}: {str(output)[:100]}...")
        if output is None:
            logger.error(f"Tool {tool_name} returned None")
            self.metrics["errors"].append(f"Tool {tool_name} returned None")
            return json.dumps({"error": f"Tool {tool_name} returned None"})
        if not isinstance(output, str):
            logger.error(f"Tool {tool_name} returned non-string output: {type(output)}")
            self.metrics["errors"].append(f"Tool {tool_name} returned non-string: {type(output)}")
            return json.dumps({"error": f"Tool {tool_name} returned non-string output: {type(output)}"})
        try:
            json.loads(output)
            return output
        except json.JSONDecodeError as e:
            logger.error(f"JSONDecodeError in {tool_name} output: {str(e)}")
            self.metrics["errors"].append(f"JSONDecodeError in {tool_name}: {str(e)}")
            return json.dumps({"error": f"Invalid JSON from {tool_name}: {str(e)}"})

    def update_context(self, tool_outputs: List[Dict[str, Any]], user_prompt: str):
        for output in tool_outputs:
            data = {}
            if output.get("content") and isinstance(output["content"], str):
                try:
                    data = json.loads(output["content"])
                except json.JSONDecodeError as e:
                    logger.error(f"JSONDecodeError in parsing tool output {output['name']}: {str(e)}")
                    self.metrics["errors"].append(f"JSONDecodeError in {output['name']}: {str(e)}")
                    continue
            if output["name"] == "get_flight_status":
                self.context.update({
                    "flight_number": data.get("flight_number"),
                    "delay_minutes": data.get("delay_minutes", 0),
                    "departure_icao": data.get("departure_icao", "KJFK"),
                    "destination_icao": data.get("destination_icao", "KLAX")
                })
            elif output["name"] == "check_crew_availability":
                self.context.update({"crew_available": data.get("available", False), "crew_data": output["content"]})
            elif output["name"] == "estimate_passenger_impact":
                self.context.update({"passenger_cost": data.get("total_cost", 0)})
            elif output["name"] == "predict_delay_escalation":
                self.context.update({
                    "escalation_likelihood": data.get("escalation_likelihood", 0.5),
                    "predicted_additional_delay": data.get("predicted_additional_delay", 30)
                })
            elif output["name"] in ["estimate_passenger_feedback", "update_passenger_feedback"]:
                self.context.update({"satisfaction_score": data.get("satisfaction_score", 0.5)})
            elif output["name"] == "get_atc_update":
                self.context.update({"atc_status": data.get("status", "No update"), "atc_delay": data.get("additional_delay_min", 0)})
            elif output["name"] == "calculate_cost_threshold":
                self.context.update({"threshold_recommendation": data.get("recommendation", "Delay")})

        # Improved priority detection
        prompt_lower = user_prompt.lower()
        if re.search(r"\bprioritize\s+cost\b|\bminimize\s+cost\b|\bcost\s+effective\b|\bcost\b", prompt_lower):
            self.context["priority"] = "cost"
            logger.info("Updated priority to cost from prompt")
        elif re.search(r"\bprioritize\s+passenger\b|\bpassenger\s+satisfaction\b", prompt_lower):
            self.context["priority"] = "passenger"
            logger.info("Updated priority to passenger from prompt")
        elif re.search(r"\bprioritize\s+operation\b|\boperational\b", prompt_lower):
            self.context["priority"] = "operation"
            logger.info("Updated priority to operation from prompt")
        else:
            logger.info(f"No priority specified, retaining priority: {self.context['priority']}")

        # Update delay_minutes from prompt
        match = re.search(r"(\d+)\s*(?:minute|min)\w*(?:\s*delay)?", prompt_lower)
        if match:
            self.context["delay_minutes"] = int(match.group(1))
            logger.info(f"Updated delay_minutes to {self.context['delay_minutes']} from prompt")
        elif "delay" in prompt_lower and not self.context.get("delay_minutes"):
            self.context["delay_minutes"] = 90
            logger.info("No specific delay found in prompt, defaulting to 90 minutes")

    def reflect(self, tool_outputs: List[Dict[str, Any]], priority: str) -> str:
        logger.info("Reflecting on tool outputs for synthesis")
        priority = self.context.get("priority", priority)
        if priority not in ["cost", "passenger", "operation"]:
            logger.warning(f"Invalid priority {priority}, defaulting to passenger")
            priority = "passenger"
            self.context["priority"] = priority

        recommendation = []
        total_cost = 0
        risk_level = "medium"
        compliance_status = "Compliant"
        passenger_impact = "Neutral"
        satisfaction_score = 0.5
        audit_trail = f"Reflection at {time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())}"

        flight_number = self.context.get("flight_number", "AA123")
        delay_min = self.context.get("delay_minutes", 90)
        crew_available = self.context.get("crew_available", False)
        pax_cost = self.context.get("passenger_cost", 0)
        escalation_likelihood = self.context.get("escalation_likelihood", 0.5)
        satisfaction_score = self.context.get("satisfaction_score", 0.5)
        atc_status = self.context.get("atc_status", "No update")
        weather_conditions = "Unknown"
        alternate_routes = []

        for output in tool_outputs:
            try:
                data = json.loads(output["content"])
                if output["name"] == "get_alternate_routes":
                    alternate_routes = data.get("alternates", [])
                elif output["name"] == "check_regulatory_compliance":
                    compliance_status = data.get("status", "Compliant")
                elif output["name"] == "suggest_operations_decision":
                    recommendation = data.get("recommendation", [])
                    total_cost = float(data.get("estimated_cost", "$0.00").replace("$", ""))
                    risk_level = data.get("risk_level", "medium")
                elif output["name"] == "estimate_passenger_impact":
                    pax_cost = data.get("total_cost", 0)
                elif output["name"] == "predict_delay_escalation":
                    escalation_likelihood = data.get("escalation_likelihood", 0.5)
                elif output["name"] in ["estimate_passenger_feedback", "update_passenger_feedback"]:
                    satisfaction_score = data.get("satisfaction_score", 0.5)
                elif output["name"] == "get_aviation_weather":
                    weather_conditions = data.get("conditions", "Unknown")
            except json.JSONDecodeError as e:
                logger.error(f"JSONDecodeError in reflect for {output['name']}: {str(e)}")
                self.metrics["errors"].append(f"JSONDecodeError in {output['name']}: {str(e)}")
                continue

        actions = []
        if priority == "cost" and delay_min >= 120 and alternate_routes:
            actions.append({
                "action": f"Reroute via {alternate_routes[0]['route']} (+{alternate_routes[0]['extra_time_min']} min)",
                "priority": 1,
                "risk": "medium",
                "cost": "$3000",
                "passenger_impact": "Neutral"
            })
            actions.append({
                "action": f"Notify passengers: Reroute via {alternate_routes[0]['route']}",
                "priority": 2,
                "risk": "low",
                "cost": "$1500",
                "passenger_impact": "Neutral"
            })
            actions.append({
                "action": "Proactive connection management at LAX",
                "priority": 3,
                "risk": "medium",
                "cost": f"${pax_cost}",
                "passenger_impact": "Neutral"
            })
            total_cost = 3000 + 1500 + pax_cost
            passenger_impact = "Neutral"
        elif priority == "passenger" and delay_min > 0:
            actions.append({
                "action": f"Delay {delay_min} min and provide amenities",
                "priority": 1,
                "risk": "medium" if escalation_likelihood > 0.3 else "low",
                "cost": "$3150" if delay_min > 90 else "$900",
                "passenger_impact": "Positive"
            })
            actions.append({
                "action": f"Notify passengers: Delay {delay_min} min",
                "priority": 2,
                "risk": "low",
                "cost": "$100",
                "passenger_impact": "Positive"
            })
            actions.append({
                "action": "Monitor ATC updates",
                "priority": 3,
                "risk": "low",
                "cost": "$0",
                "passenger_impact": "Neutral"
            })
            total_cost = (3150 if delay_min > 90 else 900) + 100
            passenger_impact = "Positive"
        else:
            actions = recommendation if recommendation else [{
                "action": f"Delay {delay_min} min and monitor",
                "priority": 1,
                "risk": "medium",
                "cost": "$0",
                "passenger_impact": "Neutral"
            }]
            total_cost = pax_cost
            passenger_impact = "Neutral"

        final_content = (
            f"### AOC Action Plan: Flight {flight_number} ({delay_min}-Minute Delay)\n"
            f"**Priority:** {'Cost' if priority == 'cost' else 'Passenger Satisfaction' if priority == 'passenger' else 'Operation'}\n\n"
            f"### 1. Executive Summary\n"
            f"Flight {flight_number} delayed by {delay_min} minutes due to ATC congestion. "
            f"Crew available: {crew_available}. "
            f"Passenger cost: ${pax_cost}. "
            f"Escalation likelihood: {escalation_likelihood:.2f}. "
            f"Satisfaction score: {satisfaction_score:.2f}. "
            f"Compliance: {compliance_status}, issues: {[] if compliance_status == 'Compliant' else ['Pending review']}. "
            f"ATC update: {atc_status}. "
            f"Weather: {weather_conditions}.\n"
            f"**Key Decision:** {'Reroute to minimize costs' if priority == 'cost' and delay_min >= 120 else 'Manage delay with passenger focus' if priority == 'passenger' else 'Monitor and delay'}\n\n"
            f"### 2. Prioritized Action Plan\n"
        )
        for i, action in enumerate(actions, 1):
            final_content += (
                f"#### Action {i}: {action['action']}\n"
                f"* **Priority:** {action['priority']}\n"
                f"* **Risk:** {action['risk']}\n"
                f"* **Cost:** {action['cost']}\n"
                f"* **Passenger Impact:** {action['passenger_impact']}\n"
            )
        final_content += (
            f"\n### 3. Regulatory Compliance\n"
            f"Flight remains compliant with no issues. "
            f"Proactive communication ensures DOT compliance.\n\n"
            f"### 4. Assumptions\n"
            f"Delay duration: {delay_min} minutes. "
            f"Cost estimates based on tool outputs and priority: {priority}. "
            f"Audit trail: {audit_trail}\n"
        )

        logger.info("Reflection completed")
        return final_content

    def run_query(self, user_prompt: str, max_iterations: int = 2) -> str:
        if ZAI_KEY_API == "PLACEHOLDER_ERROR_KEY":
            logger.error("API Key not loaded")
            return "Cannot run agent: API Key not loaded."

        self.conversation_history.append({"role": "user", "content": user_prompt})
        start_time = time.time()
        final_content = "No recommendation generated due to processing error."
        visualization_data = None
        priority = self.context.get("priority", "passenger")

        for iteration in range(max_iterations):
            logger.info(f"Starting iteration {iteration + 1}")
            print(f"AOC Agent: Iteration {iteration + 1} - Planning...")

            try:
                response = self.client.chat.completions.create(
                    model="glm-4.6",
                    messages=self.conversation_history,
                    tools=tools,
                    tool_choice="auto",
                )
                response_message = response.choices[0].message
                self.conversation_history.append(response_message)
                logger.info("Planning completed")
            except Exception as e:
                self.metrics["errors"].append(f"API call failed: {str(e)}")
                logger.error(f"API call failed: {str(e)}")
                self.conversation_history.append({"role": "assistant", "content": f"API error: {str(e)}. Attempting fallback."})
                continue

            tool_outputs = []
            if response_message.tool_calls:
                print("AOC Agent: Executing tools...")
                self.metrics["tools_called"] += len(response_message.tool_calls)
                for tool_call in response_message.tool_calls:
                    func_name = tool_call.function.name
                    func = available_functions.get(func_name)
                    if func:
                        try:
                            args = json.loads(tool_call.function.arguments)
                            output = func(**args)
                            output = self.validate_tool_output(output, func_name)
                            tool_outputs.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": func_name,
                                "content": output
                            })
                            self.conversation_history.append(tool_outputs[-1])
                            print(f"  - {func_name} executed: {output[:100]}...")
                            logger.info(f"Tool {func_name} executed with args: {args}")
                        except Exception as e:
                            error_msg = json.dumps({"error": f"Tool {func_name} failed: {str(e)}."})
                            self.metrics["errors"].append(f"Tool {func_name} failed: {str(e)}")
                            logger.error(f"Tool {func_name} failed: {str(e)}")
                            self.conversation_history.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": func_name,
                                "content": error_msg
                            })

            self.update_context(tool_outputs, user_prompt)

            required_tools = [
                ("get_flight_status", {"flight_number": self.context.get("flight_number", "AA123")}),
                ("get_aviation_weather", {"icao_code": self.context.get("departure_icao", "KJFK")}),
                ("check_crew_availability", {"flight_number": self.context.get("flight_number", "AA123")}),
                ("estimate_passenger_impact", {"flight_number": self.context.get("flight_number", "AA123"), "delay_minutes": self.context.get("delay_minutes", 90)}),
                ("predict_delay_escalation", {"flight_number": self.context.get("flight_number", "AA123"), "delay_minutes": self.context.get("delay_minutes", 90)}),
                ("check_regulatory_compliance", {
                    "flight_data": self.validate_tool_output(get_flight_status(self.context.get("flight_number", "AA123")), "get_flight_status"),
                    "crew_data": self.validate_tool_output(check_crew_availability(self.context.get("flight_number", "AA123")), "check_crew_availability")
                }),
                ("estimate_passenger_feedback", {"flight_number": self.context.get("flight_number", "AA123"), "delay_minutes": self.context.get("delay_minutes", 90)}),
                ("get_atc_update", {"flight_number": self.context.get("flight_number", "AA123")}),
                ("calculate_cost_threshold", {
                    "flight_data": self.validate_tool_output(get_flight_status(self.context.get("flight_number", "AA123")), "get_flight_status"),
                    "passenger_data": self.validate_tool_output(estimate_passenger_impact(self.context.get("flight_number", "AA123"), self.context.get("delay_minutes", 90)), "estimate_passenger_impact"),
                    "prediction_data": self.validate_tool_output(predict_delay_escalation(self.context.get("flight_number", "AA123"), self.context.get("delay_minutes", 90)), "predict_delay_escalation")
                }),
                ("pre_plan_crew_swap", {
                    "flight_number": self.context.get("flight_number", "AA123"),
                    "crew_data": self.validate_tool_output(check_crew_availability(self.context.get("flight_number", "AA123")), "check_crew_availability"),
                    "delay_minutes": self.context.get("delay_minutes", 90)
                }),
                ("get_alternate_routes", {
                    "flight_number": self.context.get("flight_number", "AA123"),
                    "departure_icao": self.context.get("departure_icao", "KJFK"),
                    "destination_icao": self.context.get("destination_icao", "KLAX")
                })
            ]
            executed_tools = [t["name"] for t in tool_outputs]
            for tool, args in required_tools:
                if tool not in executed_tools and self.context.get("flight_number"):
                    try:
                        output = available_functions[tool](**args)
                        output = self.validate_tool_output(output, tool)
                        tool_outputs.append({
                            "tool_call_id": f"manual_{tool}",
                            "role": "tool",
                            "name": tool,
                            "content": output
                        })
                        self.conversation_history.append(tool_outputs[-1])
                        self.metrics["tools_called"] += 1
                        print(f"  - {tool} executed (manual): {output[:100]}...")
                        logger.info(f"Manually executed tool {tool} with args: {args}")
                    except Exception as e:
                        self.metrics["errors"].append(f"Manual {tool} failed: {str(e)}")
                        logger.error(f"Manual {tool} failed: {str(e)}")

            if "suggest_operations_decision" not in executed_tools and self.context.get("flight_number"):
                try:
                    flight_data = self.validate_tool_output(get_flight_status(self.context.get("flight_number", "AA123")), "get_flight_status")
                    weather_data = self.validate_tool_output(get_aviation_weather(self.context.get("departure_icao", "KJFK")), "get_aviation_weather")
                    crew_data = self.validate_tool_output(check_crew_availability(self.context.get("flight_number", "AA123")), "check_crew_availability")
                    route_data = self.validate_tool_output(get_alternate_routes(self.context.get("flight_number", "AA123"), self.context.get("departure_icao", "KJFK"), self.context.get("destination_icao", "KLAX")), "get_alternate_routes")
                    passenger_data = self.validate_tool_output(estimate_passenger_impact(self.context.get("flight_number", "AA123"), self.context.get("delay_minutes", 90)), "estimate_passenger_impact")
                    prediction_data = self.validate_tool_output(predict_delay_escalation(self.context.get("flight_number", "AA123"), self.context.get("delay_minutes", 90)), "predict_delay_escalation")
                    compliance_data = self.validate_tool_output(check_regulatory_compliance(flight_data, crew_data), "check_regulatory_compliance")
                    feedback_data = self.validate_tool_output(estimate_passenger_feedback(self.context.get("flight_number", "AA123"), self.context.get("delay_minutes", 90)), "estimate_passenger_feedback")
                    atc_data = self.validate_tool_output(get_atc_update(self.context.get("flight_number", "AA123")), "get_atc_update")
                    threshold_data = self.validate_tool_output(calculate_cost_threshold(flight_data, passenger_data, prediction_data), "calculate_cost_threshold")

                    decision_output = suggest_operations_decision(
                        flight_data, weather_data, crew_data, route_data, passenger_data,
                        prediction_data, compliance_data, feedback_data, atc_data, threshold_data, priority
                    )
                    tool_outputs.append({
                        "tool_call_id": "manual_suggest_operations_decision",
                        "role": "tool",
                        "name": "suggest_operations_decision",
                        "content": decision_output
                    })
                    self.conversation_history.append(tool_outputs[-1])
                    self.metrics["tools_called"] += 1
                    print(f"  - suggest_operations_decision executed (manual): {decision_output[:100]}...")
                    logger.info(f"Manually executed suggest_operations_decision")
                except Exception as e:
                    self.metrics["errors"].append(f"Manual suggest_operations_decision failed: {str(e)}")
                    logger.error(f"Manual suggest_operations_decision failed: {str(e)}")

            print("AOC Agent: Reflecting & synthesizing...")
            try:
                reflect_content = self.reflect(tool_outputs, priority)
                self.conversation_history.append({"role": "assistant", "content": reflect_content})
                if "recommendation" in reflect_content.lower() or "action" in reflect_content.lower():
                    final_content = reflect_content
                    logger.info("Synthesis completed with recommendation")
                    break
            except Exception as e:
                self.metrics["errors"].append(f"Reflection failed: {str(e)}")
                logger.error(f"Reflection failed: {str(e)}")
                self.conversation_history.append({
                    "role": "assistant",
                    "content": f"Reflection error: {str(e)}. Attempting fallback."
                })

        self.metrics["response_time"] = time.time() - start_time
        final_content += f"\nMetrics: {self.metrics}"
        logger.info(f"Query completed in {self.metrics['response_time']:.2f} seconds")
        return final_content

if __name__ == "__main__":
    agent = AOCCAgent()
    prompt = "Flight AA123 is delayed by 45 minutes, prioritize passenger satisfaction. Provide a prioritized action plan with risks and costs."
    print(agent.run_query(prompt))
    follow_up = "Now, what if the delay worsens to 120 minutes? Update the plan with alternate routes, notify passengers, and prioritize cost."
    print(agent.run_query(follow_up))

AOC Agent: Iteration 1 - Planning...
AOC Agent: Executing tools...
  - get_flight_status executed: {"flight_number": "AA123", "status": "delayed", "delay_minutes": 45, "eta": "2025-10-01T18:30:00Z", ...
  - check_crew_availability executed: {"available": true, "pilot": "Available (ID: P001, Rest: 10h)", "copilot": "Available (ID: C001, Res...
  - estimate_passenger_impact executed: {"flight_number": "AA123", "passengers": 192, "rebooking_cost": 0, "compensation_cost": 0, "total_co...
  - predict_delay_escalation executed: {"escalation_likelihood": 0.325, "predicted_additional_delay": 30}...
  - get_aviation_weather executed (manual): {"icao": "KJFK", "metar": "KJFK 011730Z 27010KT 10SM FEW030 BKN050 15/10 A2992", "conditions": "Ligh...
  - check_regulatory_compliance executed (manual): {"status": "Compliant", "issues": []}...
  - estimate_passenger_feedback executed (manual): {"flight_number": "AA123", "satisfaction_score": 0.675, "feedback": "Neutral"}...
  - get_atc_update executed (